In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import shutil
import fiona
from fiona._drivers import GDALEnv
from shapely.geometry import Point, Polygon
import contextily as ctx
from ipyleaflet import Map, GeoData, basemaps, LayersControl, Popup, WidgetControl, GeoJSON
from ipywidgets import Text, HTML
import json



gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


liccont = gpd.read_file('C:/Users/sladea1/liccontpop.tab',driver="MapInfo File")
unliccont = gpd.read_file('C:/Users/sladea1/unliccontpop.tab',driver="MapInfo File")
allpolys = gpd.read_file('C:/Users/sladea1/Desktop/meet/Tier5_Niveau5_Jan82020.TAB',driver="MapInfo File")
Tier4polys =  gpd.read_file('C:/Users/sladea1/Desktop/meet/t4polys.tab',driver="MapInfo File")
#gl10 = gpd.read_file('p:/gl10.KML', driver = "KML")
#gl10 = gl10.geometry.unary_union

allpolys.rename(columns={"Service_Area_Zone_de_service": "area", "Service_Area_Name": "name", "Population_2016":"pop2016"}, inplace = True)
Tier4polys.rename(columns={"AREA_LABEL": "area", "Area_name_E": "name", "Pop_2016":"pop2016"}, inplace=True)
liccont.rename(columns={"LICENSEE_COMPANY":"licensee"}, inplace=True)
unliccont.rename(columns={"Operator_Name": "licensee", "Centre_Frequency__GHz_":"Frequency_Min__MHz_","Licensed_Bandwidth__MHz_":"Frequency_Max__MHz_"}, inplace=True)




In [ ]:
Tier5_data = GeoData(geo_dataframe = allpolys,
                   style={'color': 'red', 'fillColor' : 'white','weight':1, 'dashArray':'2', 'fillOpacity':0.1},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Tier5')

Tier4_data = GeoData(geo_dataframe = Tier4polys,
                   style={'color': 'black', 'fillColor' : 'white','weight':1, 'lineArray':'2', 'fillOpacity':0.1},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'Tier4')


liccont_data = GeoData(geo_dataframe = liccont,
                   style={'color': 'blue', 'fill': 'white','weight':1, 'lineArray':'1', 'fillOpacity':0.0},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.5},
                   name = 'Licenced interference contours')

unliccont_data = GeoData(geo_dataframe = unliccont,
                   style={'color': 'orange', 'fill': 'white','weight':1, 'dashArray':'1', 'fillOpacity':0.0},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.2},
                   name = 'Unlicenced interference contours')

#gl10_data = GeoData(geo_dataframe = gl10)


In [ ]:
from ipyleaflet import FullScreenControl, SearchControl
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets

def update_html(feature,  **kwargs):
    html.value = '''
        <h4><b>{}-{}</b></h4>
        <p>Population: {:,.1f}</p>
        <p>% encumbered by Licenced spectrum: {:0.2%}</p>
        <p>% encumbered by Unicenced spectrum: {:0.2%}</p>
    '''.format(feature['properties']['area'],
               feature['properties']['name'],
               feature['properties']['pop2016'],
               feature['properties']['lencumpop'],
               feature['properties']['uencumpop'])

def update1_html(feature,  **kwargs):
    html2.value = '''
        <h5><b>{}</b></h5>
        <p>Population encumbered: {:,.2f}</p>
        <p>Min Freq: {:0.2f} Max Freq: {:0.2f}</p>
    '''.format(feature['properties']['licensee'],
               feature['properties']['gpopencum'],
               feature['properties']['Frequency_Min__MHz_'],
               feature['properties']['Frequency_Max__MHz_'])
    
    
    
def loadgriddata(feature, **kwargs):
    fn = "P:/Tier4/{}.tab".format(feature['properties']['area'])
    gridfreq = gpd.read_file(fn,driver="MapInfo File")
    grid_data = GeoData(geo_dataframe = gridfreq,
                   style={'color': 'blue', 'fill': 'white','weight':1, 'lineArray':'1', 'fillOpacity':0.0},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.5},
                   name = 'Gridcells')
    m.add_layer(grid_data)
 



center = (55.204793, -100.121558)    
m = Map(basemap=basemaps.OpenStreetMap.BlackAndWhite, center=center, zoom=5)



m.add_layer(Tier5_data);
m.add_layer(Tier4_data);
m.add_layer(liccont_data);
m.add_layer(unliccont_data);
#m.add_layer(gl10_data);

# Popup with a given location on the map:

m.add_control(LayersControl())

m.add_control(FullScreenControl())


html = HTML('''Hover on a Tier''')
html2 = HTML('''Contour''')
html.layout.margin = '0px 20px 20px 20px'
html2.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
control1 = WidgetControl(widget=html2, position='bottomright')
m.add_control(control)
m.add_control(control1)


Tier5_data.on_hover(update_html)

Tier4_data.on_hover(update_html)
#Tier4_data.on_click(loadgriddata)
#Tier4_data.on_click(centremap1)

liccont_data.on_hover(update1_html)

unliccont_data.on_hover(update1_html)


m






